In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
#载入数据集
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

#设定批次大小
batch_size=128
#计算一共有多少批次
n_batch=mnist.train.num_examples//batch_size

#定义placeholder
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

keep_prob=tf.placeholder(tf.float32)
#创建一个神经网络
w=tf.Variable(tf.truncated_normal([784,2000],stddev=0.1))
b=tf.Variable(tf.zeros([2000])+0.1)
h1=tf.nn.tanh(tf.matmul(x,w)+b)
h1_drop=tf.nn.dropout(h1,keep_prob)

w1=tf.Variable(tf.truncated_normal([2000,2000],stddev=0.1))
b1=tf.Variable(tf.zeros([2000])+0.1)
h2=tf.nn.tanh(tf.matmul(h1,w1)+b1)
h2_drop=tf.nn.dropout(h2,keep_prob)

w2=tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b2=tf.Variable(tf.zeros([1000])+0.1)
h3=tf.nn.tanh(tf.matmul(h2,w2)+b2)
h3_drop=tf.nn.dropout(h3,keep_prob)

w3=tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b3=tf.Variable(tf.zeros([10])+0.1)

prediction=tf.nn.softmax(tf.matmul(h3_drop,w3)+b3)
#定义loss函数
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))


#train_step=tf.train.GradientDescentOptimizer(0.2).minimize(loss)
#train_step=tf.train.MomentumOptimizer(0.2,momentum=0.2).minimize(loss)
train_step=tf.train.AdamOptimizer(1e-2).minimize(loss)#Adam一般用比较小的学习率，但收敛比SGD快

#初始化
init=tf.global_variables_initializer()

#定义准确率
#argmax找ont-hot编码里最大的那个数的索引，两个比较，相等就是预测对了，反之
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(100):
        for batch in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        test_acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:0.7})
        
        train_acc=sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:0.7})
        
        print('iter'+str(epoch)+',testing accuracy:'+str(test_acc)+' training acc:'+str(train_acc))